In [18]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split, GridSearchCV ,KFold
import pandas as pd
from sklearn.metrics import ConfusionMatrixDisplay,confusion_matrix
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.calibration import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [19]:
df = pd.read_csv('/Users/shahadaleissa/Downloads/Code/Data/colon-dataset-processed.csv')
le = LabelEncoder()
df['Class'] = le.fit_transform(df['Class'])
df

,Class,Age,p16540_C/C,p16540_G/C,p16540_G/G,p16580_C/C,p16580_C/T,p16580_T/T,mdm2_G/G,mdm2_G/T,mdm2_T/T,GAL3_A/A,GAL3_C/A,GAL3_C/C,TIM1_C/C,TIM1_G/C,TIM1_G/G
0,1,49,False,True,False,True,False,False,True,False,False,False,True,False,False,True,False
1,1,49,False,True,False,True,False,False,True,False,False,True,False,False,False,True,False
2,1,49,False,True,False,True,False,False,False,True,False,False,True,False,True,False,False
3,1,36,False,True,False,True,False,False,False,False,True,False,True,False,True,False,False
4,1,49,True,False,False,False,True,False,False,False,True,False,False,True,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,0,81,False,True,False,True,False,False,False,False,True,False,False,True,False,False,True
111,0,73,True,False,False,True,False,False,False,False,True,False,False,True,False,False,True
112,0,56,True,False,False,True,False,False,False,True,False,False,False,True,False,True,False
113,0,74,False,True,False,False,True,False,True,False,False,False,False,True,False,False,True


In [20]:
X=df.drop('Class',axis=1)
y=df['Class']

In [21]:

#split data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
random_state=123

In [22]:
#overampling
sm = SMOTE(random_state=random_state)
X_oversampled, y_oversampled = sm.fit_resample(X, y)

X_train_oversampled, X_test_oversampled, y_train_oversampled, y_test_oversampled = train_test_split(X_oversampled, y_oversampled, test_size=0.2, random_state=123)


In [23]:
#underampling
rus = RandomUnderSampler(random_state=random_state)
X_undersampled, y_undersampled = rus.fit_resample(X, y)
X_train_undersampled, X_test_undersampled, y_train_undersampled, y_test_undersampled = train_test_split(X_undersampled, y_undersampled, test_size=0.2, random_state=123)

In [24]:

def model_training(X_train, X_test, y_train, y_test, random_state=42):
    # List of models to train
    models = [
        AdaBoostClassifier(),
        ExtraTreesClassifier(random_state=random_state, n_jobs=-1), # Parallel processing enabled
        HistGradientBoostingClassifier(random_state=random_state),
        LGBMClassifier(),
        LogisticRegression(random_state=random_state, n_jobs=-1), # Parallel processing enabled
        RandomForestClassifier(max_depth=10, random_state=random_state, n_estimators=100, n_jobs=-1), # Parallel processing enabled
        SVC(random_state=random_state, kernel='linear', C=1),
        XGBClassifier(random_state=random_state)
    ]

    # Create an empty DataFrame to store results
    results_df = pd.DataFrame(columns=['Model', 'Accuracy'])

    for model in models:
        # Train the model
        model.fit(X_train, y_train)
        
        # Make predictions
        predictions = model.predict(X_test)
        
        # Evaluate the model
        accuracy = accuracy_score(y_test, predictions)
        precision =precision_score(y_test, predictions)
        recall= recall_score(y_test, predictions)
        cm=confusion_matrix(y_test, predictions)
        
        # Append results to the DataFrame
        temp_df = pd.DataFrame({'Model': [model.__class__.__name__], 'Accuracy': [accuracy], 'Precision': [precision], 'recall': [recall], 'cm':[cm]})
        results_df = pd.concat([results_df, temp_df], ignore_index=True)


    # Sort and return the results DataFrame
    return results_df.sort_values(by=['Accuracy'], ascending=False)


In [25]:
def cm(df, ada_loc, et_loc, hgb_loc):

    print("Adaboost confusion matrix")
    print(df['cm'].loc[ada_loc])

    print("ExtraTrees confusion matrix")
    print(df['cm'].loc[et_loc])

    print("HGB confusion matrix")
    print(df['cm'].loc[hgb_loc])

In [26]:
original=model_training(X_train, X_test, y_train, y_test)

/var/folders/k4/_ms9gszn46s_6f86x4tywvbc0000gn/T/ipykernel_4929/3089428379.py:32: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, temp_df], ignore_index=True)


[LightGBM] [Info] Number of positive: 40, number of negative: 52
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000564 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 38
[LightGBM] [Info] Number of data points in the train set: 92, number of used features: 9
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.434783 -> initscore=-0.262364
[LightGBM] [Info] Start training from score -0.262364
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain

/opt/homebrew/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [27]:
original

,Model,Accuracy,Precision,recall,cm
0,AdaBoostClassifier,0.869565,1.000000,0.7,"[[13, 0], [3, 7]]"
4,LogisticRegression,0.826087,0.800000,0.8,"[[11, 2], [2, 8]]"
6,SVC,0.826087,0.800000,0.8,"[[11, 2], [2, 8]]"
1,ExtraTreesClassifier,0.782609,0.727273,0.8,"[[10, 3], [2, 8]]"
2,HistGradientBoostingClassifier,0.782609,0.727273,0.8,"[[10, 3], [2, 8]]"
3,LGBMClassifier,0.782609,0.727273,0.8,"[[10, 3], [2, 8]]"
7,XGBClassifier,0.782609,0.727273,0.8,"[[10, 3], [2, 8]]"
5,RandomForestClassifier,0.739130,0.700000,0.7,"[[10, 3], [3, 7]]"


In [28]:
cm(original,0,1,2)

Adaboost confusion matrix
[[13  0]
 [ 3  7]]
ExtraTrees confusion matrix
[[10  3]
 [ 2  8]]
HGB confusion matrix
[[10  3]
 [ 2  8]]


In [29]:
oversample= model_training(X_train_oversampled, X_test_oversampled, y_train_oversampled, y_test_oversampled)

/var/folders/k4/_ms9gszn46s_6f86x4tywvbc0000gn/T/ipykernel_4929/3089428379.py:32: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, temp_df], ignore_index=True)


[LightGBM] [Info] Number of positive: 50, number of negative: 54
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000485 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 40
[LightGBM] [Info] Number of data points in the train set: 104, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.480769 -> initscore=-0.076961
[LightGBM] [Info] Start training from score -0.076961
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best ga

In [30]:
oversample

,Model,Accuracy,Precision,recall,cm
0,AdaBoostClassifier,0.884615,0.928571,0.866667,"[[10, 1], [2, 13]]"
1,ExtraTreesClassifier,0.884615,0.875000,0.933333,"[[9, 2], [1, 14]]"
2,HistGradientBoostingClassifier,0.846154,0.866667,0.866667,"[[9, 2], [2, 13]]"
6,SVC,0.846154,0.866667,0.866667,"[[9, 2], [2, 13]]"
7,XGBClassifier,0.846154,0.866667,0.866667,"[[9, 2], [2, 13]]"
3,LGBMClassifier,0.807692,0.857143,0.800000,"[[9, 2], [3, 12]]"
4,LogisticRegression,0.807692,0.857143,0.800000,"[[9, 2], [3, 12]]"
5,RandomForestClassifier,0.807692,0.857143,0.800000,"[[9, 2], [3, 12]]"


In [31]:
cm(oversample,0,1,2)

Adaboost confusion matrix
[[10  1]
 [ 2 13]]
ExtraTrees confusion matrix
[[ 9  2]
 [ 1 14]]
HGB confusion matrix
[[ 9  2]
 [ 2 13]]


In [32]:
undersample= model_training(X_train_undersampled, X_test_undersampled, y_train_undersampled, y_test_undersampled)

/var/folders/k4/_ms9gszn46s_6f86x4tywvbc0000gn/T/ipykernel_4929/3089428379.py:32: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, temp_df], ignore_index=True)


[LightGBM] [Info] Number of positive: 39, number of negative: 41
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003670 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 36
[LightGBM] [Info] Number of data points in the train set: 80, number of used features: 8
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.487500 -> initscore=-0.050010
[LightGBM] [Info] Start training from score -0.050010
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain

In [33]:
undersample

,Model,Accuracy,Precision,recall,cm
2,HistGradientBoostingClassifier,0.95,0.916667,1.000000,"[[8, 1], [0, 11]]"
3,LGBMClassifier,0.95,0.916667,1.000000,"[[8, 1], [0, 11]]"
4,LogisticRegression,0.95,0.916667,1.000000,"[[8, 1], [0, 11]]"
0,AdaBoostClassifier,0.90,0.909091,0.909091,"[[8, 1], [1, 10]]"
5,RandomForestClassifier,0.90,0.846154,1.000000,"[[7, 2], [0, 11]]"
6,SVC,0.90,0.909091,0.909091,"[[8, 1], [1, 10]]"
7,XGBClassifier,0.90,0.846154,1.000000,"[[7, 2], [0, 11]]"
1,ExtraTreesClassifier,0.80,0.769231,0.909091,"[[6, 3], [1, 10]]"


In [34]:
cm(undersample,0,1,2)

Adaboost confusion matrix
[[ 8  1]
 [ 1 10]]
ExtraTrees confusion matrix
[[ 6  3]
 [ 1 10]]
HGB confusion matrix
[[ 8  1]
 [ 0 11]]
